In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
path = 'drive/My Drive/data'

In [5]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import tensorflow as tf
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [6]:
df = pd.read_csv('/content/drive/MyDrive/datathon/sorted_filtered_source_data.csv')

In [7]:
df.shape


(4680360, 14)

In [8]:
df

,customer_id,day_no,usage_app_facebook_daily,usage_app_helakuru_daily,usage_app_other,usage_app_tiktok_daily,usage_app_whatsapp_daily,usage_app_youtube_daily,usage_pack_data,usage_pack_vas,usage_voice_d2d_incoming,usage_voice_d2d_outgoing,usage_voice_d2nd_outgoing,usage_voice_nd2d_incoming
0,0000f727854b50bb95c054b39c1fe5c92e5ebcfa4bcb5d...,1,60.31,32.38,159.70,39.77,12.23,0.00,100.52,28.93,36.91,55.62,34.91,10.47
1,0000f727854b50bb95c054b39c1fe5c92e5ebcfa4bcb5d...,2,0.00,58.27,56.24,280.53,6.47,242.45,99.60,15.20,6.43,67.57,16.74,16.54
2,0000f727854b50bb95c054b39c1fe5c92e5ebcfa4bcb5d...,3,133.73,12.05,135.03,168.41,10.51,274.21,98.73,30.40,28.52,98.07,17.92,15.46
3,0000f727854b50bb95c054b39c1fe5c92e5ebcfa4bcb5d...,4,262.71,0.00,120.87,100.44,9.33,0.00,99.64,20.88,23.28,215.60,27.30,19.80
4,0000f727854b50bb95c054b39c1fe5c92e5ebcfa4bcb5d...,5,96.26,27.85,118.67,492.71,10.04,313.87,99.94,18.70,11.89,0.00,37.04,16.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680355,fffff7f18e3f2477c5c981222df6260c01b0e9324cfc87...,86,277.94,48.34,303.31,0.00,0.00,261.67,48.31,0.00,32.74,43.46,0.00,0.00
4680356,fffff7f18e3f2477c5c981222df6260c01b0e9324cfc87...,87,329.72,49.20,451.45,0.00,11.65,228.95,0.00,0.00,58.11,11.80,0.00,0.00
4680357,fffff7f18e3f2477c5c981222df6260c01b0e9324cfc87...,88,341.92,48.99,413.73,0.00,0.00,223.03,154.70,0.00,32.52,9.40,0.00,0.00
4680358,fffff7f18e3f2477c5c981222df6260c01b0e9324cfc87...,89,337.44,50.05,341.92,0.00,0.00,237.96,259.29,0.00,8.91,71.29,0.00,0.00


In [9]:
# melted_df = pd.melt(df, id_vars=['customer_id', 'daily_usage'], var_name='day_no', value_name='data_usage')

# pivoted_df = melted_df.pivot_table(index=['customer_id', 'day_no'], columns='daily_usage', values='data_usage', aggfunc='first')

# pivoted_df = pivoted_df.reset_index()


# pivoted_df.columns.name = None
# pivoted_df.columns = ['customer_id', 'day_no'] + [f"{app}" for app in pivoted_df.columns[2:]]

# pivoted_df.head(15)

# melted_df = pd.melt(df, id_vars=['customer_id', 'daily_usage'], var_name='day_no', value_name='data_usage')

# # Pivot the melted dataframe to get application names as columns
# pivoted_df = melted_df.pivot_table(index=['customer_id', 'day_no'], columns='daily_usage', values='data_usage', aggfunc='first')

# # Reset index to bring 'customer_id' and 'day_no' back as columns
# pivoted_df = pivoted_df.reset_index()

# # Rename the columns for application names
# pivoted_df.columns.name = None
# pivoted_df.columns = ['customer_id', 'day_no'] + [f"{app}" for app in pivoted_df.columns[2:]]

# pivoted_df['day_no'] = pivoted_df['day_no'].astype(int)
# pivoted_df = pivoted_df.sort_values(by=['customer_id', 'day_no'])

# # Reset index again after sorting
# pivoted_df = pivoted_df.reset_index(drop=True)

# # Assuming 'pivoted_df' is the final transformed dataframe
# pivoted_df.head(20)

melted_df = pd.melt(df, id_vars=['customer_id', 'daily_usage'], var_name='day_no', value_name='data_usage')

pivoted_df = melted_df.pivot_table(index=['customer_id', 'day_no'], columns='daily_usage', values='data_usage', aggfunc='first')

pivoted_df = pivoted_df.reset_index()

pivoted_df.columns.name = None
pivoted_df.columns = ['customer_id', 'day_no'] + [f"{app}" for app in pivoted_df.columns[2:]]
pivoted_df['day_no'] = pivoted_df['day_no'].str.extract('(\d+)').astype(int)
pivoted_df = pivoted_df.sort_values(by=['customer_id', 'day_no'])

pivoted_df = pivoted_df.reset_index(drop=True)
print(pivoted_df.head(20))



KeyError: ignored

In [ ]:
pivoted_df

In [10]:
missing_values = df.isna().sum()
print(missing_values)
missing_percentages = (df.isnull().sum() / len(df)) * 100
print(missing_percentages)

customer_id                  0
day_no                       0
usage_app_facebook_daily     0
usage_app_helakuru_daily     0
usage_app_other              0
usage_app_tiktok_daily       0
usage_app_whatsapp_daily     0
usage_app_youtube_daily      0
usage_pack_data              0
usage_pack_vas               0
usage_voice_d2d_incoming     0
usage_voice_d2d_outgoing     0
usage_voice_d2nd_outgoing    0
usage_voice_nd2d_incoming    0
dtype: int64
customer_id                  0.0
day_no                       0.0
usage_app_facebook_daily     0.0
usage_app_helakuru_daily     0.0
usage_app_other              0.0
usage_app_tiktok_daily       0.0
usage_app_whatsapp_daily     0.0
usage_app_youtube_daily      0.0
usage_pack_data              0.0
usage_pack_vas               0.0
usage_voice_d2d_incoming     0.0
usage_voice_d2d_outgoing     0.0
usage_voice_d2nd_outgoing    0.0
usage_voice_nd2d_incoming    0.0
dtype: float64


In [ ]:
customer_ids = pivoted_df['customer_id']
days = pivoted_df['day_no']
data_without_id = pivoted_df.drop(columns=['customer_id','day_no'])

imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data_without_id)

imputed_df = pd.DataFrame(data_imputed, columns=data_without_id.columns)
imputed_df.insert(0, 'customer_id', customer_ids)
#imputed_df.insert(1, 'day_no', days) This one left as a comment because at first we did not convert the day values to numerical values
imputed_df.to_csv('imputed_data.csv', index=False)

In [ ]:
df.dtypes

In [11]:
customer_id_column = df['customer_id']
day_no_column = df['day_no']
columns_to_normalize = df.columns.difference(['customer_id','day_no'])

# Extract the columns you want to normalize using double square brackets
features = df[columns_to_normalize]

# Create a StandardScaler instance
scaler = MinMaxScaler()

# Fit the scaler on your feature data to compute mean and standard deviation
scaler.fit(features)

# Transform the features to have zero mean and unit variance
normalized_features = scaler.transform(features)
scaled_data = scaler.fit_transform(features)

# Create a new DataFrame with normalized features
normalized_data = pd.DataFrame(normalized_features, columns=columns_to_normalize)

# Combine the customer_id column with the normalized features
normalized_data = pd.concat([customer_id_column,day_no_column, normalized_data], axis=1)

In [12]:
normalized_data

,customer_id,day_no,usage_app_facebook_daily,usage_app_helakuru_daily,usage_app_other,usage_app_tiktok_daily,usage_app_whatsapp_daily,usage_app_youtube_daily,usage_pack_data,usage_pack_vas,usage_voice_d2d_incoming,usage_voice_d2d_outgoing,usage_voice_d2nd_outgoing,usage_voice_nd2d_incoming
0,0000f727854b50bb95c054b39c1fe5c92e5ebcfa4bcb5d...,1,0.034228,0.127541,0.068761,0.025969,0.047972,0.000000,0.042677,0.061548,0.072048,0.126616,0.076843,0.022298
1,0000f727854b50bb95c054b39c1fe5c92e5ebcfa4bcb5d...,2,0.000000,0.229518,0.024215,0.183182,0.025379,0.085401,0.042287,0.032338,0.012551,0.153820,0.036848,0.035225
2,0000f727854b50bb95c054b39c1fe5c92e5ebcfa4bcb5d...,3,0.075897,0.047463,0.058139,0.109969,0.041225,0.096589,0.041917,0.064675,0.055671,0.223252,0.039445,0.032925
3,0000f727854b50bb95c054b39c1fe5c92e5ebcfa4bcb5d...,4,0.149098,0.000000,0.052042,0.065586,0.036597,0.000000,0.042304,0.044422,0.045442,0.490803,0.060092,0.042168
4,0000f727854b50bb95c054b39c1fe5c92e5ebcfa4bcb5d...,5,0.054631,0.109697,0.051095,0.321732,0.039382,0.110558,0.042431,0.039784,0.023209,0.000000,0.081532,0.035672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4680355,fffff7f18e3f2477c5c981222df6260c01b0e9324cfc87...,86,0.157741,0.190405,0.130595,0.000000,0.000000,0.092171,0.020511,0.000000,0.063908,0.098935,0.000000,0.000000
4680356,fffff7f18e3f2477c5c981222df6260c01b0e9324cfc87...,87,0.187128,0.193792,0.194379,0.000000,0.045697,0.080646,0.000000,0.000000,0.113430,0.026862,0.000000,0.000000
4680357,fffff7f18e3f2477c5c981222df6260c01b0e9324cfc87...,88,0.194052,0.192965,0.178138,0.000000,0.000000,0.078561,0.065680,0.000000,0.063478,0.021399,0.000000,0.000000
4680358,fffff7f18e3f2477c5c981222df6260c01b0e9324cfc87...,89,0.191510,0.197140,0.147219,0.000000,0.000000,0.083820,0.110086,0.000000,0.017392,0.162288,0.000000,0.000000


In [13]:
scaled_data

array([[0.03422815, 0.12754057, 0.06876122, ..., 0.12661628, 0.0768435 ,
        0.02229794],
       [0.        , 0.22951788, 0.02421497, ..., 0.15381989, 0.0368479 ,
        0.03522522],
       [0.07589671, 0.04746337, 0.05813918, ..., 0.22325168, 0.0394453 ,
        0.03292514],
       ...,
       [0.19405221, 0.19296518, 0.17813763, ..., 0.02139865, 0.        ,
        0.        ],
       [0.19150965, 0.19714038, 0.14721877, ..., 0.16228829, 0.        ,
        0.        ],
       [0.13864926, 0.20029148, 0.19311268, ..., 0.0874613 , 0.08298481,
        0.        ]])

In [14]:
scaled_data.shape

(4680360, 12)

In [15]:
features = scaled_data

In [16]:
targets = pd.read_csv('/content/drive/MyDrive/datathon/sorted_train_data.csv')

In [17]:
target = targets['churn_status']

In [18]:
target.shape

(52004,)

In [19]:
i = features.shape[0] // 90
reshaped_features = features.reshape(i, 90, 12)
reshaped_features.shape

(52004, 90, 12)

In [20]:
from sklearn.model_selection import train_test_split

In [21]:
x_train, x_test, y_train, y_test = train_test_split(reshaped_features, target, test_size=0.20, random_state=123, shuffle=False)

In [118]:
y_train_arr=np.array(y_train)
y_test_arr = np.array(y_test)

In [83]:
x_train.shape

(41603, 90, 12)

In [24]:
reshaped_x_train = x_train.reshape(3744270, 12)

In [82]:
reshaped_x_train

array([0.03422815, 0.12754057, 0.06876122, 0.02596919, 0.04797207,
       0.        , 0.04267731, 0.06154795, 0.07204763, 0.12661628,
       0.0768435 , 0.02229794])

In [26]:
x_test.shape

(10401, 90, 12)

In [27]:
reshaped_x_test = x_test.reshape(936090, 12)

In [85]:
reshaped_x_test[0]

array([0.04031215, 0.12765874, 0.        , 0.13412301, 0.14391622,
       0.0695715 , 0.10625597, 0.04027317, 0.03015811, 0.04329812,
       0.1214836 , 0.        ])

In [106]:
def find_y_values(arr_len,strike,data_array):
  array_with_value = np.full(arr_len,2)

  for i in range(len(data_array)):

    c= strike+i*strike

    if c<len(array_with_value):
      array_with_value[c] = data_array[i]
      #print(array_with_value[c],data_array[i])

  return array_with_value

In [116]:
train_test_lables = find_y_values(len(reshaped_x_train),90,y_train_arr)

In [119]:
test_test_lables = find_y_values(len(reshaped_x_test),90,y_test_arr)

In [124]:
test_test_lables[191]

2

In [34]:
# i = 90
# s = 90
# iteration = 90 * 41603 / 90
# length = 90 * 41603

In [35]:
# churn_array_train = np.full(length, -1)

In [38]:
# iteration = 0
# index = 0

# for x in range(0, 41603):
#   index = i + (s * iteration)

#   if index == 3744270:
#     break

#   churn_array_train[index] = y_train[iteration]
#   iteration = iteration + 1


In [ ]:
# churn_array_train[::90]

In [ ]:
# count =0
# for _ in churn_array_train:
#   if(_==0 or _==1):
#     count +=1
# print(count)

In [41]:
# i = 90
# s = 90
# len_test = 90*10401
# churn_array_test = np.full(len_test, -1)

In [ ]:
# churn_array_test

In [ ]:
# y_test

In [47]:
# iteration = 41603
# index = 0

# for x in range(0, 10401):
#   index = i + (s * (iteration - 41603))

#   if index == 936090:
#     break

#   churn_array_test[index] = y_test[iteration]
#   iteration = iteration + 1

In [48]:
# print(len(churn_array_train))

3744270


In [49]:
# reshaped_x_train.shape

(3744270, 12)

In [134]:
win_length = 90
batch_size = 1000
num_features = 12

train_generator = TimeseriesGenerator(reshaped_x_train,train_test_lables , length = win_length, sampling_rate=1, batch_size=batch_size, stride=90)
test_generator = TimeseriesGenerator(reshaped_x_test, test_test_lables, length=win_length, sampling_rate=1, batch_size=batch_size, stride=90)

In [ ]:
train_generator[1]

In [136]:
train_test_lables[:90]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2])

In [137]:
import tensorflow as tf

model = tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(1200, input_shape=(win_length,num_features), return_sequences=True))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.LSTM(1200, return_sequences=True))
model.add(tf.keras.layers.ReLU())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.LSTM(600, return_sequences=False))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))


In [138]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 90, 1200)          5822400   
                                                                 
 re_lu_4 (ReLU)              (None, 90, 1200)          0         
                                                                 
 lstm_13 (LSTM)              (None, 90, 1200)          11524800  
                                                                 
 re_lu_5 (ReLU)              (None, 90, 1200)          0         
                                                                 
 dropout_8 (Dropout)         (None, 90, 1200)          0         
                                                                 
 lstm_14 (LSTM)              (None, 600)               4322400   
                                                                 
 dropout_9 (Dropout)         (None, 600)              

In [139]:
model.compile(loss='binary_crossentropy', optimizer=tf.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
hisotry = model.fit_generator(train_generator, epochs=50, validation_data=test_generator, shuffle=False)

Epoch 1/50


<ipython-input-139-3ea24a72684b>:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hisotry = model.fit_generator(train_generator, epochs=50, validation_data=test_generator, shuffle=False)


42/42 [==============================] - 49s 1s/step - loss: 0.3888 - accuracy: 0.8566 - val_loss: 0.1221 - val_accuracy: 0.9523
Epoch 2/50
42/42 [==============================] - 44s 1s/step - loss: 0.0433 - accuracy: 0.9853 - val_loss: 0.0413 - val_accuracy: 0.9845
Epoch 3/50
42/42 [==============================] - 44s 1s/step - loss: 0.0370 - accuracy: 0.9874 - val_loss: 0.0358 - val_accuracy: 0.9867
Epoch 4/50
42/42 [==============================] - 44s 1s/step - loss: 0.0883 - accuracy: 0.9735 - val_loss: 0.0535 - val_accuracy: 0.9844
Epoch 5/50
42/42 [==============================] - 44s 1s/step - loss: 0.0396 - accuracy: 0.9866 - val_loss: 0.0335 - val_accuracy: 0.9882
Epoch 6/50
42/42 [==============================] - 44s 1s/step - loss: 0.0347 - accuracy: 0.9881 - val_loss: 0.0307 - val_accuracy: 0.9892
Epoch 7/50
42/42 [==============================] - 44s 1s/step - loss: 0.0313 - accuracy: 0.9893 - val_loss: 0.0274 - val_accuracy: 0.9907
Epoch 8/50
42/42 [=============

In [140]:
test_data = pd.read_csv('/content/drive/MyDrive/datathon/sorted_filtered_test_source_data.csv')


In [141]:
customer_id_column = test_data['customer_id']
day_no_column = test_data['day_no']
columns_to_normalize = test_data.columns.difference(['customer_id','day_no'])

# Extract the columns you want to normalize using double square brackets
features = test_data[columns_to_normalize]

# Create a StandardScaler instance
scaler = MinMaxScaler()

# Fit the scaler on your feature data to compute mean and standard deviation
scaler.fit(features)

# Transform the features to have zero mean and unit variance
normalized_features = scaler.transform(features)
scaled_data = scaler.fit_transform(features)



In [143]:
scaled_data.shape

(1170090, 12)

In [147]:
scaled_data[0]

array([0.04481355, 0.07390598, 0.16735366, 0.        , 0.        ,
       0.02893026, 0.        , 0.03668237, 0.06372681, 0.04572412,
       0.04766904, 0.04059129])

In [150]:
b = scaled_data.shape[0]//90
reshaped_scaled_data = scaled_data.reshape(b,90,12)

prediction = model.predict(reshaped_scaled_data)

407/407 [==============================] - 9s 22ms/step


In [152]:
binary_predictions = (prediction > 0.5).astype(int)

In [156]:
ground_truth = pd.read_csv('/content/drive/MyDrive/datathon/sorted_ground_truth.csv')

In [158]:
ground_truth_values = ground_truth['churn_status']

In [163]:
flatten = binary_predictions.flatten()

In [175]:
flatten[:50]

array([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0])

In [174]:
ground_truth_values.values[:50]

array([0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0])

In [176]:
accuracy = np.mean(flatten == ground_truth_values.values)

In [177]:
accuracy

0.9954618875471117